<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Higher Analysis**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

2.2.0


## **Differentialgleichung**

#### **Funktionalanalysis**

**Analysis**

Die Begriffe der Norm (als Verallgemeinerung des Betrags), der Konvergenz, der Stetigkeit und der Grenzwerte lassen sich einfach von einer in mehrere Dimensionen verallgemeinern.

https://de.m.wikipedia.org/wiki/Analysis

* Gewöhnliche Differentialgleichungen, Eindimensionale Differential- & Integralrechnung

* Functional analysis is organized around adequate techniques to bring function spaces as topological vector spaces within reach of the ideas that would apply to normed spaces of finite dimension. 

* Die historischen Wurzeln der Funktionalanalysis liegen im Studium der **Fouriertransformation** und aahnlicher Transformationen sowie der Untersuchung von Differential- und Integralgleichungen. 

* Der Wortbestandteil ”funktional“ geht auf die **Variationsrechnung** zuru ̈ck.

* Grundlage der Funktionalanalysis sind Vektorräume über den reellen oder komplexen Zahlen. Der Grundbegriff ist hier der topologische Vektorraum, der dadurch gekennzeichnet ist, **dass die Vektorraumverknüpfungen stetig sind**, etwas konkreter werden auch lokalkonvexe topologische Vektorräume und Fréchet-Räume untersucht. 

* Wichtige Aussagen sind dabei der Satz von Hahn-Banach, der Satz von Baire und der Satz von Banach-Steinhaus. Insbesondere in der Lösungstheorie partieller Differentialgleichungen spielen diese eine wichtige Rolle, darüber hinaus in der Fredholm-Theorie.

* Der wichtigste Spezialfall **lokalkonvexer topologischer Vektorräume** sind **normierte Vektorräume**. Sind diese zusätzlich vollständig, dann heißen sie **Banachräume**. Noch spezieller betrachtet man **Hilberträume**, bei denen die Norm von einem Skalarprodukt erzeugt wird. Diese Räume sind von grundlegender Bedeutung für die mathematische Formulierung der Quantenmechanik. Ein wichtiger Untersuchungsgegenstand sind stetige lineare Operatoren auf Banach- oder Hilberträumen.



Folgende Themen sollen behandelt werden:

* Distributionen
* Unbeschränkte Operatoren, Spektralsatz 
* Banach-Räume, Hilbert-Räume (insbesondere Lp- und **Sobolev Räume**) und beschränkte lineare Operatoren darauf.
* Pseudodifferentialoperatoren
* Grundlegende Sätze der Funktionalanalysis (Hahn-Banach, Banach-Steinhaus, Satz vom abgeschlossenen Graphen...)
* Kompakte Operatoren und Fredholm Theorie
* Spektraltheorie kompakter Operatoren
* Grundzüge nicht beschränkter Operatoren
* Anwendungen auf Randwertprobleme elliptischer partieller Differentialgleichungen


Der große Reiz der Funktionalanalysis ist, dass die im Grundstudium erworbenen Konzepte der Analysis (Stetigkeit, Grenzwerte, Vollständigkeit) und Linearen Algebra (Vektorräume, lineare Abbildungen, Eigenwerte) konsequent zusammengeführt werden und gemeinsam eine Theorie liefern, die wiederrum in vielen Bereichen der Mathematik und Physik ein unabdingbares Werkzeug ist. Im Rahmen des Masterschwerpunktes Partielle Differentialgleichungen sollen in dieser Vorlesung insbesondere die Anwendung im Studium partieller Differentialgleichungen im Fokus liegen.


**In der Funktionalanalysis betrachtet man topologische Vektorräume, d. h. Vektorräume, auf denen man von Konvergenz sprechen und dementsprechend auch unendliche Summen bilden kann**. Good overview [here](https://matheplanet.com/default3.html?call=article.php?sid=1192&ref=https%3A%2F%2Fwww.google.de%2F)

**Das Ziel der Funktionalanalysis** ist es nicht zuletzt, die **<u>Methoden der mehrdimensionalen Analysis auf unendlichdimensionale Räume auszudehnen und dabei insbesondere Konzepte wie Konvergenz, Stetigkeit und Differenzierbarkeit zu untersuchen</u>**. 

Daher werden a priori **nur Vektorräume betrachtet, die zumindest eine topologische Struktur tragen, also die topologischen Vektorräume**. Zu ihnen zählen unter anderem alle normierten Vektorräume und insbesondere die Banach- und Hilberträume.

https://de.m.wikipedia.org/wiki/Nichtstandardanalysis

#### **Ableitungen**

Die Ableitung ist zunächst nur für einen Punkt $\left(x_{0} \mid f\left(x_{0}\right)\right)$ auf dem Graphen einer Funktion $f(x)$ bzw. für eine Stelle $x_{0}$ definiert. Sie ist gegeben durch:

> $\lim _{x \rightarrow x_{0}} \frac{f(x)-f\left(x_{0}\right)}{x-x_{0}}$ oder alternativ $\lim _{h \rightarrow 0} \frac{f\left(x_{0}+h\right)-f\left(x_{0}\right)}{h}$

Anschaulich erhält man durch den [Differenzenquotienten](https://de.serlo.org/mathe/1669/differenzenquotient) die Steigung der Sekante an den Funktionsgraphen von dem Punkt x
0
 und einem anderen Punkt. Die Idee bei beiden Defintionen ist, dass man die Tangente durch Sekanten annähert, indem man den x-Wert des zweiten Punktes immer näher an x
0
 wählt. 

![ff](https://raw.githubusercontent.com/deltorobarba/repo/master/ableitung.png)

*Drittes Bild mit A = B: Die Steigung dieser Tangente ist gleich der Ableitung an der Stelle x0*



Jede differenzierbare Funktion ist stetig, die Umkehrung gilt jedoch nicht.

**Alternative Notationen**

$f^{\prime}(x)=\frac{\mathrm{d} f}{\mathrm{~d} x}(x)=\frac{\mathrm{d} f(x)}{\mathrm{d} x}=\frac{\mathrm{d}}{\mathrm{d} x} f(x)$

https://de.serlo.org/mathe/1795/ableitung

> Der **Differentenquotient** $\lim _{h \rightarrow 0} \frac{f\left(x_{0}+h\right)-f\left(x_{0}\right)}{h}$ gibt die Steigung durch 2 Punkte an. Mit lim fur h gegen Null im Differentenquotienten ergibt es den **Differentialquotient** f<sup>'</sup> (x<sub>0</sub>) (erste Ableitung, das ist die Steigung an einer Stelle).

> Notation mit dem [Differentialoperator](https://www.youtube.com/watch?v=LjeFmpkoUxQ&list=PLLTAHuUj-zHhJSdV8hqWaydgOWjF-fYK0&index=3): $\frac{\mathrm{d}}{\mathrm{~d} x}$ sieht folgendermassen aus: $f^{\prime}\left(x\right)$ = $\frac{\mathrm{d}}{\mathrm{~d} x}\left(f(x)\right)$

**Mehrfache Ableitungen**

* Erste Ableitung gibt die Steigung einer Funktion
* Zweite Ableitung: Wann ist die Steigung konstant?
* Zum Beispiel ist die erste Ableitung des Orts x(t) nach der Zeit t die Momentangeschwindigkeit, die zweite Ableitung die Beschleunigung.

**Von der Ableitung zur Ableitungsfunktion (Die Ableitung als Funktion mit dem Differenzenquotient)**

Von der Ableitung an einem bestimmten Punkt ist es nur ein kleiner Schritt zur Ableitung auf dem ganzen Definitionsbereich. Denn alle "normalen" Funktionen (die in der Schule behandelt werden) besitzen für alle $x \in D$ den gleichen Differenzenquotienten in Abhängigkeit von $x$. In dem Fall kann man die Ableitungen mit Hilfe der Ableitungsregeln bestimmen.

Beispiel mit $f(x)=x^{2}$

Wir berechnen die Ableitung der Funktkon $f(x)=x^{2}$ und benutzen die Definition $\lim _{x \rightarrow x_{0}} \frac{f(x)-f\left(x_{0}\right)}{x-x_{0}} .$ Mit Hilfe der dritten binomischen Formel bekommen wir für die Ableitung $\lim _{x \rightarrow x_{0}} \frac{x^{2}-x_{0}^{2}}{x-x_{0}}=\lim _{x \rightarrow x_{0}} \frac{\left(x+x_{0}\right)\left(x-x_{0}\right)}{x-x_{0}}=\lim _{x \rightarrow x_{0}}\left(x+x_{0}\right)=2 x_{0} .$ Das
heißt, dass egal an welchem Punkt $x_{0}$ wir die Ableitung ausrechnen, wir bekommen immer $2 x_{0}$

* **Eine Ableitungsfunktion (kurz: Ableitung) ist eine Funktion, die jeder Stelle x0 den Wert ihres Differentialquotienten zuordnet.**

* Die Ableitungsfunktion zu einer gegebenen Funktion f ist – vereinfacht ausgedrückt – die Tangentensteigungsfunktion. Das bedeutet: An jeder Stelle x hat die Steigung der Tangente an den Graphen von f einen bestimmten Wert, welcher der Ableitung der Funktion entspricht. Nun kannst du quasi mit einem Lineal den Graphen von f "abfahren". Das Lineal soll dabei immer tangential an den Graphen anliegen. Wenn du dann die Steigung des Lineals Punkt für Punkt in ein Diagramm überträgst, erhältst du ein Bild der Ableitungsfunktion. Dabei musst du die Funktionsgleichung selbst nicht kennen, sondern nur den Graphen.

**Definition: Differenzierbarkeit**

Eine Funktion $f: U \rightarrow \mathbb{R},$ die ein **offenes Intervall** $U$ in die reellen Zahlen abbildet, heißt differenzierbar an der Stelle $x_{0} \in U$, falls der Grenzwert

> $
\lim _{x \rightarrow x_{0}} \frac{f(x)-f\left(x_{0}\right)}{x-x_{0}}=\lim _{h \rightarrow 0} \frac{f\left(x_{0}+h\right)-f\left(x_{0}\right)}{h} \quad\left(\text { mit } h=x-x_{0}\right)
$

existiert. Dieser Grenzwert heißt Differentialquotient oder Ableitung von $f$ nach $x$ an der Stelle $x_{0}$ und wird notiert als

> $f^{\prime}\left(x_{0}\right)$ oder $\left.\frac{\mathrm{d} f(x)}{\mathrm{d} x}\right|_{x=x_{0}}$ oder $\frac{\mathrm{d} f}{\mathrm{~d} x}\left(x_{0}\right)$ oder $\frac{\mathrm{d}}{\mathrm{d} x} f\left(x_{0}\right)$

**Eine weitere Definition ist**: Es gibt eine an der Stelle $x_{0}$ stetige Funktion $r$ mit $r\left(x_{0}\right)=0$ und eine Konstante $L$, sodass für alle $x$ gilt

> $
f(x)=f\left(x_{0}\right)+L\left(x-x_{0}\right)+r(x)\left(x-x_{0}\right)
$

Die Bedingungen $r\left(x_{0}\right)=0$ und dass $r$ an der Stelle $x_{0}$ stetig ist, bedeuten gerade, dass das " Restglied" $r(x)$ für $x$ gegen $x_{0}$ gegen 0 konvergiert.

**Schwache Ableitung**

* Eine [schwache Ableitung](https://de.wikipedia.org/wiki/Schwache_Ableitung) ist in der Funktionalanalysis, einem Teilgebiet der Mathematik, eine Erweiterung des Begriffs der gewöhnlichen (klassischen) Ableitung. Er ermöglicht es, Funktionen eine Ableitung zuzuordnen, die nicht (stark bzw. im klassischen Sinne) differenzierbar sind.

* Schwache Ableitungen spielen eine große Rolle in der Theorie der partiellen Differentialgleichungen. Räume schwach differenzierbarer Funktionen sind die Sobolev-Räume. Ein noch allgemeinerer Begriff der Ableitung ist die Distributionenableitung.

* In mathematics, a weak derivative is a generalization of the concept of the derivative of a function (strong derivative) **for functions not assumed differentiable, but only integrable**, i.e., to lie in the $L^{p}$ space $L^{1}([a, b])$. See distributions for a more general definition.

**Glatte Funktionen**

* Eine [glatte Funktion](https://de.m.wikipedia.org/wiki/Glatte_Funktion) ist eine mathematische Funktion, die unendlich oft differenzierbar (insbesondere stetig) ist

* Zum Beispiel ist jede [holomorphe Funktion](https://de.m.wikipedia.org/wiki/Holomorphe_Funktion) (=in jedem Punkt komplex differenzierbar) auch eine glatte Funktion. Außerdem werden glatte Funktionen als Abschneidefunktionen oder als Testfunktionen für Distributionen verwendet.

* Eine Funktion $f: D \rightarrow \mathbb{R}$ heißt unendlich oft (stetig) differenzierbar oder glatt, wenn $f \in C^{n}(D)$ für alle $n \in \mathbb{N}$ gilt. Die Menge aller glatten Funktionen auf $D$ wird mit $C^{\infty}(D)$ notiert und es gilt
$
C^{\infty}(D):=\bigcap_{n \in \mathbb{N}} C^{n}(D)
$

#### **Differentialgleichungen**

* Eine Differentialgleichung ist eine mathematische **Gleichung für eine gesuchte Funktion** von einer oder mehreren Variablen, **in der auch Ableitungen dieser Funktion vorkommen**. Differentialgleichungen sind also Gleichungen, in denen 
  * **Ableitungen** einer 
  * **unbekannten Funktion** auftauchen.

* Differentialgleichungen werden überall dort verwendet, wo die Änderung einer Größe von der gleichen Größe selbst abhängt. Diese Funktion nutzt man zur Modellierung physikalischer Vorgänge wie Wachstum, Bewegung oder Kräfte. Beispiele:

  * Die Funktion f beschreibt den Ort, dann beschreibt die f´ die Änderung des Ortes und das ist nichts anderes, als die Geschwindigkeit. 

  * Die Funktion f beschreibt die Größe eine Bevölkerung, dann beschreibt f´deren Änderung und das ist nichts anderes als das Bevölkerungswachstum

  * Allgemein beschreibt die Funktion f eine Größe und f´die Änderungsrate dieser Größe.

* Die **Lösung einer Differentialgleichung** kann im Allgemeinen nicht durch die Gleichung selbst eindeutig bestimmt werden, sondern benötigt zusätzlich noch weitere Anfangs- oder Randwerte zu exakten Bestimmung. Daher ist es nicht möglich, eine allgemein gültige Lösungsmethodik anzugeben. Nur für gewöhnliche, integrable Differentialgleichungen existiert ein allgemeines Lösungsverfahren.
Folgende Lösungsverfahren sind möglich:

  * Für gewöhnliche Differentialgleichungen benutzt man die **Umkehrung des Differenzierens**, in dem man die Stammfunktion aufsucht und so die Differentialgleichung integriert. Die Lösungsfunktion ist dann einfach die Stammfunktion der Differentialgleichung. Beispiel: f´(x) = 4, dann ist die Stammfunktion F(x) = 4x + C und somit die Lösung der Differentialgleichung.

  * Partielle Differentialgleichungen werden in erster Linie durch Trennung der Variablen und spätere Integration gelöst.

  * Zum Beispiel durch (n-fache) Integration fur f(x) = x<sup>2</sup> bestimmt man das unbestimmte Integral $y=\int f(x) \mathrm{d} x+C$ (Siehe [hier](https://www.lern-online.net/mathematik/analysis/differentialrechnung/differentialgleichung/differentialgleichung-1-ordnung-loesen/) und [hier](https://www.youtube.com/watch?v=wUwxsEbEwHA)).

  * **Aber Achtung: ein mathematisches Modell (stetig, differenzierbar) kann die Realität oft nur unzureichend widerspiegeln. Beispiel ist die Zinseszinsrechnung. Oder bei Computern: Ein kontinuierlicher Vorgang wird diskret modelliert, z. B. bei der Simulation von Bewegungen am Computer, da der Computer nur in diskreten Schritten rechnen kann.** ([Source](https://www.spektrum.de/spektrum/projekt2/gaes3.htm))

* **Beispiel: Bewegungsgleichung**

  * **Gegeben** sind die Beschleunigung = zweifache Ableitung

  * **Gesucht** (unbekannte Funktion): Gesucht ist eine Gleichung x(t), die den Ort angibt, an dem sich der fallende Stein zu einer bestimmten Zeit befindet.

  * **Lösung**: Integration, und zwar zweifach - Umkehrung der zweifachen Differentiation (hierbei in Gestalt einer Differentialgleichung zweiter Ordnung)
  
  * Hierbei kann man aus einer gegebenen Beschleunigung auf die Ortsfunktion eines Körpers zurückzuschließen, also wo befindet sich ein fallendes Objekt zum Zeitpunkt t? Man kennt nur Masse und Beschleunigung. Die mathematische Schwierigkeit dieses Problems rührt daher, dass Ort, Geschwindigkeit und Beschleunigung Vektoren sind, die im Allgemeinen nicht in die gleiche Richtung zeigen, und dass die Kraft von der Zeit t und vom Ort ${\vec {s}}$ abhängen kann.

* Siehe auch [weitere Beispiele einfacher Differentialgleichungen aus der Natur](https://www.spektrum.de/spektrum/projekt2/gaes3.htm):

  * Radioaktiver Zerfall

  * Zinsrechnung

* Bei Differentialgleichungen bekommt man dann als Loesung auch eine Funktion und keine Zahl: **Die Lösung einer Differentialgleichung ist immer eine Funktion (oder im Falle eines Systems von Differentialgleichungen mehrere Funktionen). Es ist jedoch nicht jede Differentialgleichung lösbar.** [Source](https://mathepedia.de/Gewoehnliche_Differentialgleichungen.html)

* In den Differentialgleichungen werden Funktionen nicht mit f(x), sondern mit y(x) bezeichnet

* Da viele Modelle mehrdimensional sind, sind bei der Formulierung häufig die partiellen Ableitungen sehr wichtig, mit denen sich **partielle Differentialgleichungen** formulieren lassen. Mathematisch kompakt werden diese mittels **Differentialoperatoren** beschrieben und analysiert.

https://www.lernort-mint.de/mathematik/ableitung/differentialgleichungen-verwendung/

https://www.th-nuernberg.de/fileadmin/fakultaeten/amp/Stry/kapitel10.pdf

https://avhschule.de/differentialgleichungen/

https://de.wikipedia.org/wiki/Differentialrechnung

#### **Gewöhnliche Differentialgleichungen** 

Eine (gewöhnliche) Differentialgleichung erster Ordnung hat die Form 

> $y^{\prime}=f(x, y)$ (explizite Form) 

> $F\left(x, y, y^{\prime}\right)=0$ (implizite Form)

(Achtung: y steht hierbei fur eine ganze Funktion, nicht nur einen y-Wert!)

Der einfachste Typ einer solchen Differentialgleichung hat die Gestalt:

> $
y^{\prime}=f(x)
$

hängt also nur von $x$ und nicht von $y$ ab. Diese Differentialgleichung kann durch Integrieren gelöst werden. Die allgemeine Lösung ergibt sich dann mit $y=\int f(x) \mathrm{d} x+C$. [Source](https://mathepedia.de/DGL_Erster_Ordnung.html)

Ist die Differentialgleichung nach der **höchsten Ableitung aufgelöst**, so nennt man die gewöhnliche Differentialgleichung **explizit**, d. h., es gilt:

> $y^{(n)}(x)=f\left(x, y(x), y^{\prime}(x), \ldots, y^{(n-1)}(x)\right)$ bzw.

> $y^{(n)}=f\left(x, y, y^{\prime}, \ldots, y^{(n-1)}\right)$

Funktionsgraphen werden in der Regel durch eine Gleichung y=f(x) beschrieben und sind deswegen explizit dargestellte Kurven

In den Anwendungen sind explizite gewöhnliche Differentialgleichungen mathematisch einfacher zu verarbeiten. Die höchste vorkommende Ableitungsordnung n wird Ordnung der Differentialgleichung genannt. ([Source](https://de.m.wikipedia.org/wiki/Gewöhnliche_Differentialgleichung))

Eine **explizite** gewöhnliche Differentialgleichung 1. Ordnung hat die Gestalt: 

> $y^{\prime}(x)=f(x, y(x))$

**Implizite gewöhnliche Differentialgleichung**

Eine gewöhnliche Differentialgleichung für die gesuchte Funktion $y(x)$ heisst **implizit**, wenn sie in der Form ist:

> $F\left(x, y(x), y^{\prime}(x), \ldots, y^{(n)}(x)\right)=0$

* Die implizite Differentiation (auch implizite Ableitung) ist eine Möglichkeit, **eine Funktion, die nicht explizit durch einen Term, sondern nur implizit durch eine Gleichung gegeben ist** (auch implizite Kurve), mit Hilfe der mehrdimensionalen Differentialrechnung abzuleiten. 
* Sie kann oft auch benutzt werden, um die Ableitung von Funktionen, die zwar explizit gegeben sind, **in dieser Form aber schwierig abzuleiten sind**, zu bestimmen.
* Eine implizite Kurve ist in der Mathematik eine Kurve in der euklidischen Ebene, die durch eine Gleichung der Form F(x, y) = 0
* Eine implizite Kurve ist also die Gesamtheit der Nullstellen einer Funktion von zwei Variablen. 
* Implizit bedeutet, dass die Gleichung der Kurve nicht nach x oder y aufgelöst ist.
* Siehe auch: [Implizite_Kurve](https://de.wikipedia.org/wiki/Implizite_Kurve) und [Implizite Differentiation](https://de.wikipedia.org/wiki/Implizite_Differentiation)

**Explizite gewöhnliche Differentialgleichung**

Funktionsgraphen werden in der Regel durch eine Gleichung y=f(x) beschrieben und sind deswegen explizit dargestellte Kurven


Ist die Differentialgleichung nach der **höchsten Ableitung aufgelöst**, so nennt man die gewöhnliche Differentialgleichung **explizit**, d. h., es gilt:

> $y^{(n)}(x)=f\left(x, y(x), y^{\prime}(x), \ldots, y^{(n-1)}(x)\right)$ bzw.

> $y^{(n)}=f\left(x, y, y^{\prime}, \ldots, y^{(n-1)}\right)$


In den Anwendungen sind explizite gewöhnliche Differentialgleichungen mathematisch einfacher zu verarbeiten. Die höchste vorkommende Ableitungsordnung n wird Ordnung der Differentialgleichung genannt. 

Eine **explizite** gewöhnliche Differentialgleichung 1. Ordnung hat die Gestalt: 

> $y^{\prime}(x)=f(x, y(x))$

Extrempunkte: erste Ableitung ist Null.

**Implizit vs explizit**

* Eine Funktion kann in Form 𝑦=𝑓(𝑥) dargestellt werden. Das bedeutet, dass diese Funktion nach der Variablen 𝑦 aufgelöst ist. Man spricht in diesem Fall von einer expliziten Darstellung.

* Da bei dieser Darstellungsform jedem 𝑥-Wert nur ein 𝑦-Wert zugeordnet wird, können einfache und häufig notwendige Kurven wie Ellipsen oder Kreislinien nicht geschlossen dargestellt werden (zu jedem 𝑥-Wert existieren mehrere 𝑦-Werte), d.h. es wird zur Darstellung mehr als eine Funktion benötigt. Damit für die Darstellung von Ellipsen oder Kreisen nicht mehrere Funktionen aufgestellt werden müssen, kann man diese auch in impliziter Form angeben.

* Implizite Darstellung: Bei der impliziten Darstellungsform ist die Funktion nicht nach einer der beiden Variablen aufgelöst. Die Funktion wird als Menge aller Nullstellen von 𝐹 angegeben und hat die Form:

> $𝐹 (𝑥,𝑦)=0$

* Mit Hilfe der impliziten Darstellung können auch Ellipsen und Kreise dargelegt werden. Anwendungsbeispiele: Um einen Kreis abzubilden, bedarf es bei der expliziten Darstellung zwei Funktionen. 

https://www.ingenieurkurse.de/hoehere-mathematik-analysis-gewoehnliche-differentialgleichungen/darstellungsarten-ebener-kurven/implizite-und-explizite-darstellung.html

**Kurvenschar (Funktionsschar)**

* Die Lösungsmenge einer gewöhnlichen Differentialgleichung ist im allgemeinen eine Funktionenschar.

* Eine [Kurvenschar](https://de.wikipedia.org/wiki/Kurvenschar), auch [Funktionenschar](https://mathepedia.de/Kurvenscharen.html), Funktionsschar oder Parameterfunktion, ist eine Menge verschiedener Kurven, deren Abbildungsvorschriften sich in mindestens einem Parameter unterscheiden. Sonderfälle sind das Büschel, eine einparametrige Schar, und das Bündel, eine Schar mit einem allen Funktionen gemeinsamen Punkt.

![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Kurvenschar_parabel.svg/360px-Kurvenschar_parabel.svg.png)

*Parabelschar $f_{a}(x)=a x^{2}$. Bündel in (0,0) und Büschel in $a$*


#### **Partielle Differentialgleichung**

* Partielle Differenzen werden angegeben mit $\partial$

* Erlauterung in [Youtube](https://www.youtube.com/watch?v=ly4S0oi3Yz8)

* Ω als Teilmenge von $R$<sup>n</sup> ist als Gebiet offen und zusammenhängend

* Offene Mengen sind wichtig fur die Bildung einer Ableitung (weil man ja keinen rand mochte, sondern an jedem Punkt der Menge differenzieren, einen Differenzenquotienten bilden). Man braucht genügend Punkte in der Umgebung - man kann eine Kugel um jeden Punkt legen, der komplett in der Menge Ω liegt (siehe auch Definition von offen in einem metrischen Raum). Offenheitsbedingungen verwenden, um Grenzwerte zu definieren, u.a. von diesen Differenzenquotienten

* Zusammenhängend: wählen wir 2 Punkte aus der Menge, finden wir immer einen direkten Weg zwischen ihnen (man kann eine stetige Abbildung vom Intervall Punkt 1 zu Punkt 2 bilden (muss keine Gerade sein).

**Eine partielle Differentialgleichung in Ω ist eine Bedingung in der Form:**

> $F\left(x, u, \left(\partial^{\alpha} u\right)_{1 \leqslant|\alpha| \leq m}\right)=0$

> $(x \in \Omega, u: \Omega \rightarrow \mathbb{R}, \quad \alpha \quad$ Multindex $)$

**Eine klassische Losung der partiellen Differentialgleichung (PDE) ist:**

> $u: \Omega \rightarrow \mathbb{R} \quad, \quad u \in C^{m}(\Omega)$ mit

> $\forall_{x \in \Omega} \quad F\left(x, u(x), \quad\left(\partial^{\alpha} u(x)\right)_{1 \leq |\alpha| \leq m}\right)$

**Partielle Differentialgleichungen werden in erster Linie durch Trennung der Variablen und spätere Integration gelöst.**

Die Zahl m heisst die Ordnung der PDE, wenn sie minimal gewahlt ist.

https://www.youtube.com/watch?v=vBxifpKoVwc&t=25s

**Lineare PDE auf Ω (Gebiet)**

Sobolevräume sind ein grundlegendes Werkzeug bei der Behandlung von PDG (rein und angewandt).

See also **Variational Formulations** - wichtig fur partielle Differentialgleichungen

https://de.wikipedia.org/wiki/Partielle_Differentialgleichung

**Euler-Gleichungen**

* Euler-Gleichungen (Strömungsmechanik)

* bildet dann ein System von **nichtlinearen partiellen** Differentialgleichungen **erster Ordnung**

* Wird die Viskosität vernachlässigt ($\eta =\lambda =0$), so erhält man die Euler-Gleichungen (hier für den kompressiblen Fall)

> $\rho \frac{\partial \vec{v}}{\partial t}+\rho(\vec{v} \cdot \nabla) \vec{v}=-\nabla p+\vec{f}$

https://de.wikipedia.org/wiki/Euler-Gleichungen_(Strömungsmechanik)

**Navier-Stokes-Gleichungen**

* Die Gleichungen sind eine **Erweiterung der Euler-Gleichungen** der Strömungsmechanik **um Viskosität beschreibende Terme** (Die Navier-Stokes-Gleichungen beinhalten die Euler-Gleichungen als den Sonderfall, in dem die innere Reibung (Viskosität) und die Wärmeleitung des Fluids vernachlässigt werden.)

* bildet dann ein System von **nichtlinearen partiellen** Differentialgleichungen **zweiter Ordnung**

* Die Navier-Stokes-Gleichungen bilden das Verhalten von Wasser, Luft und Ölen ab und werden daher in diskretisierter Form bei der Entwicklung von Fahrzeugen wie Autos und Flugzeugen angewendet. 

* Dies geschieht in Näherungsform, da keine exakten analytischen Lösungen für diese komplizierten Anwendungsfälle bekannt sind.

> $\rho \overrightarrow{\vec{v}}=\rho\left(\frac{\partial \vec{v}}{\partial t}+(\vec{v} \cdot \nabla) \vec{v}\right)=-\nabla p+\mu \Delta \vec{v}+(\lambda+\mu) \nabla(\nabla \cdot \vec{v})+\vec{f}$

https://de.wikipedia.org/wiki/Navier-Stokes-Gleichungen

#### **Anfangs- & Randwertproblem**

Beispielsweise werden alle schwingenden Pendel durch eine Differentialgleichung beschrieben (siehe: Pendelgleichung), und der generelle Bewegungsablauf folgt immer dem gleichen Prinzip. Der konkrete Bewegungsablauf ist jedoch durch die Rand- oder Anfangsbedingung(en) (wann wurde das Pendel angestoßen, und wie weit) bestimmt. Die Lösbarkeit von Anfangswertproblemen bei gewöhnlichen Differentialgleichungen 1. Ordnung wird durch den Satz von Picard-Lindelöf beschrieben.

https://mathepedia.de/Gewoehnliche_Differentialgleichungen.html

**Randwertproblem**

* [Randwertprobleme](https://de.m.wikipedia.org/wiki/Randwertproblem) nennt man eine **wichtige Klasse von Problemstellungen**, bei denen zu einer vorgegebenen [Differentialgleichung](https://de.m.wikipedia.org/wiki/Differentialgleichung) Lösungen gesucht werden, die auf dem **Rand des Definitionsbereiches** vorgegebene Funktionswerte (Randbedingungen) annehmen sollen. 

* Im Falle einer gewöhnlichen Differentialgleichung ist der Definitionsbereich der Funktion ein abgeschlossenes Intervall.

* Siehe auch: https://de.wikipedia.org/wiki/Dirichlet-Randbedingung

* Beim **Dirichlet-Problem** werden Funktionswerte auf dem Rand vorgegeben. Anstatt Funktionswerten werden beim **Neumann-Problem** Ableitungswerte vorgeschrieben. Die **schiefe Randbedingung** stellt eine Kombination der beiden vorangehenden Probleme dar. Hierbei soll die gesuchte Funktion auf dem Rand gleich ihrer Normalenableitung auf dem Rand sein.

* Werden hier statt zwei Werten nur an einem Randpunkt des Intervalles – meistens a – Werte für u und zusätzlich für Ableitungen von u vorgegeben, so spricht man von einem Anfangswertproblem und nennt die vorgegebenen Werte seine Anfangsbedingungen.

* Partielle Differentialgleichungen betrachtet man meistens auf Sobolew-Räumen. In diesen Räumen werden Funktionen, die bis auf Nullmengen übereinstimmen, als gleich angesehen. Da der Rand eines Gebietes üblicherweise eine Nullmenge ist, ist der Begriff der Randbedingung problematisch. Lösungen für dieses Problem sind sobolewsche Einbettungssätze oder – allgemeiner – Spuroperatoren.

* Randwertaufgaben haben nicht immer eine Lösung (siehe Beispiel unten), im Falle ihrer Existenz ist die Lösung nicht in allen Fällen eindeutig. Die Berechnung einer Näherungslösung für eine Randwertaufgabe mit Mitteln der numerischen Mathematik ist oft aufwendig und läuft meist auf die Lösung sehr großer Gleichungssysteme hinaus.

**Anfangswertprobleme (Cauchy-Problem)**

* Das Gegenstück zum Randwertproblem ist das [Anfangswertproblem](https://mathepedia.de/Anfangswertprobleme.html) auch [hier](https://de.wikipedia.org/wiki/Anfangswertproblem), bei dem die Lösung für einen beliebigen Punkt im Definitionsbereich vorgegeben wird.

* Als Anfangswertproblem bezeichnet eine **wichtige Klasse von Differentialgleichungen**. Die Lösung eines Anfangswertproblems ist die Lösung der Differentialgleichung unter zusätzlicher Berücksichtigung eines vorgegebenen Anfangswertes.

* Die Lösungsmenge einer gewöhnlichen Differentialgleichung ist im allgemeinen eine Funktionenschar. Sucht man eine spezielle Lösung (Funktion) so reicht die Vorgabe der Differentialgleichung nicht aus.

* Sind neben der Differentialgleichung $F\left(x, y, y^{\prime}, \ldots, y^{(n)}\right.$) außerdem noch Anfangsbedingungen der Form $
y\left(x_{0}\right)=c_{0}, y^{\prime}\left(x_{0}\right)=c_{1}, \ldots, y^{(n)}\left(x_{0}\right)=c_{n}$ gegeben, so spricht man von einem Anfangswertproblem.

* Beispiel: Allgemeine Lösung der Differentialgleichung $y^{\prime}=y$ ist $y=C \mathrm{e}^{x}$. Mit der Anfangsbedingung $y(0)=2$ erhalten wir die Gleichung $2=C \cdot 1$ und finden $y=2 \cdot \mathrm{e}^{x}$ als Lösung des Anfangswertproblems.

* Verallgemeinert man das Cauchy-Problem auf mehrere Veränderliche, etwa n Veränderliche $x_{1},\dotsc ,x_{n}$, so erhält man partielle Differentialgleichungen. 

Ein Anfangswertproblem **erster Ordnung** ist ein Gleichungssystem, das aus einer gewöhnlichen Differentialgleichung erster Ordnung

> $
y^{\prime}(t)=f(t, y(t))
$

und einer zusätzlichen Anfangsbedingung

> $
y\left(t_{0}\right)=y_{0}
$

besteht, mit
- dem Anfangswert $y_{0}$ und
- einem Zeitpunkt $t_{0} \in \mathbb{R}$.

Eine konkrete Funktion $y$ ist eine Lösung des Anfangswertproblems. wenn sie beide Gleichungen erfüllt. Gesucht ist also eine Funktion $y$, die die **Bedingungen der Differentialgleichung und des Anfangswertes** erfüllt.

Ist die Funktion $f$ stetig, so ist dies nach dem Hauptsatz der Integralrechnung genau
dann der Fall, wenn

> $
y(t)=y_{0}+\int_{t_{0}}^{t} f(s, y(s)) d s
$

für alle $t$ im Definitionsintervall gilt.

## **Operatoren & Spektrum**

#### **Overview**

**Operatoren sind Abbildungen zwischen Funktionenräumen (um sie von den Elementen von Funktionenräumen zu unterscheiden, die Funktionen heissen).** 

* Ahnlich wie bei Funktoren, als strukturerhaltende Abbildung zwischen zwei Kategorien, in denen selbst Funktionen existieren).

* Ein [Operator](https://de.wikipedia.org/wiki/Operator_(Mathematik)) ist eine mathematische Vorschrift, durch die man aus mathematischen Objekten neue Objekte bilden kann.

* Er kann eine standardisierte Funktion oder eine Vorschrift über Funktionen sein. 

* In der Funktionalanalysis hat man es mit Vektorräumen zu tun, deren Elemente selbst Funktionen sind. Diese Räume nennt man auch [Funktionenräume](https://de.wikipedia.org/wiki/Funktionenraum). Um die Elemente (=Funktionen) dieser Vektorräume besser von den Abbildungen zwischen solchen Vektorräumen zu unterscheiden, nennt man letztere auch Operatoren. 

* Daher: **Abbildungen zwischen solchen Vektorräumen von Funktionen / Funktionenräumen bezeichnet man auch als Operatoren.**

* Beispiele für Operatoren sind:

  * Operatoren für die Grundrechenarten: +, -, *, /

  * Funktionaloperatoren (im Sinne der Funktionalanalysis): $\frac{d}{d x} \int \nabla$ grad div rot

![cc](https://raw.githubusercontent.com/deltorobarba/repo/master/operator.png)

[Image Source](https://me-lrt.de/operator-linear-beschrankt)

**Unterschied Operator und Funktion**: 

* Was den mathematischen Formalismus angeht, gibt es keinen Unterschied zwischen einem Operator und einer Funktion (und einer Abbildung). 

* Allerdings gibt es einen 'philosophischen' Unterschied: Operatoren werden meistens für zwei Typen von Funktionen verwendet: a) Grundlegende Rechenoperationen. b) **Funktionen, deren Argumente ebenfalls Funktionen sind**. c) Funktionen in der Meta-Ebene. Mehr dazu steht im bereits verlinkten Wikipedia-Artikel.

#### **Differential- & Integraloperator**

**Differential- & Nabla-Operator**

Der [**Nabla Operator**](https://de.wikipedia.org/wiki/Nabla-Operator) $\nabla$ ist ein Symbol, das in der Vektor- und Tensoranalysis benutzt wird, **um kontextabhängig einen der drei Differentialoperatoren Gradient, Divergenz oder Rotation zu notieren**. Es ist zur Bestimmung des Gradienten einer mehrdimensionalen Funktion. Mit einem der drei Differentialoperatoren [Gradient](https://de.wikipedia.org/wiki/Gradient_(Mathematik)) (Anwendung im [Gradientenverfahren](https://de.wikipedia.org/wiki/Gradientenverfahren) in der Numerik), [Divergenz](https://de.wikipedia.org/wiki/Divergenz_eines_Vektorfeldes) oder [Rotation](https://de.wikipedia.org/wiki/Rotation_eines_Vektorfeldes) 

https://www.youtube.com/watch?v=YW-bUVIOpB0&t=51s

* **Differential**: Der [**Differentialoperator**](https://de.wikipedia.org/wiki/Differentialoperator) $\frac{\mathrm{d}}{\mathrm{d} x}$ zur Bildung von [Differentialen](https://de.wikipedia.org/wiki/Differential_(Mathematik)) (ist eine Funktion, die einer Funktion eine Funktion zuordnet und die Ableitung nach einer oder mehreren Variablen enthält.)

* [**Nabla Operator**](https://de.wikipedia.org/wiki/Nabla-Operator) $\nabla$ zur Bestimmung des Gradienten einer mehrdimensionalen Funktion. Mit einem der drei **Differentialoperatoren**.

* Der [**Differentialoperator**](https://de.wikipedia.org/wiki/Differentialoperator) $\frac{\mathrm{d}}{\mathrm{d} x}$ zur Bildung von [Differentialen](https://de.wikipedia.org/wiki/Differential_(Mathematik)) (ist eine Funktion, die einer Funktion eine Funktion zuordnet und die Ableitung nach einer oder mehreren Variablen enthält.)
  * [Gradient](https://de.wikipedia.org/wiki/Gradient_(Mathematik)): Gibt die Richtung und Stärke des steilsten Anstiegs eines Skalarfeldes an. Der Gradient eines Skalarfeldes ist ein Vektorfeld. $\operatorname{grad} \phi:=\vec{\nabla} \phi=\left(\begin{array}{c}\frac{\partial \phi}{\partial x} \\ \frac{\partial \phi}{\partial y} \\ \frac{\partial \phi}{\partial z}\end{array}\right)$
  * [Divergenz](https://de.wikipedia.org/wiki/Divergenz_eines_Vektorfeldes): Gibt die Tendenz eines Vektorfeldes an, von Punkten wegzufließen. $\operatorname{div} \vec{F}:=\vec{\nabla} \cdot \vec{F}=\frac{\partial F_{x}}{\partial x}+\frac{\partial F_{y}}{\partial y}+\frac{\partial F_{z}}{\partial z}$
  * [Rotation](https://de.wikipedia.org/wiki/Rotation_eines_Vektorfeldes): Gibt die Tendenz eines Vektorfeldes an, um Punkte zu rotieren. $\operatorname{rot} \vec{F}:=\vec{\nabla} \times \vec{F}=\left(\begin{array}{c}\frac{\partial F_{z}}{\partial y}-\frac{\partial F_{y}}{\partial z} \\ \frac{\partial F_{x}}{\partial z}-\frac{\partial F_{z}}{\partial x} \\ \frac{\partial F_{y}}{\partial x}-\frac{\partial F_{x}}{\partial y}\end{array}\right)$

* **Diese drei Rechenoperationen sind in der Vektoranalysis von besonderer Bedeutung**, weil sie Felder produzieren, die sich bei räumlicher Drehung des ursprünglichen Feldes mitdrehen. Operativ formuliert: Bei Gradient, Rotation und Divergenz spielt es keine Rolle, ob sie vor oder nach einer Drehung angewendet werden. Diese Eigenschaft folgt aus den **koordinatenunabhängigen** Definitionen. 

https://www.youtube.com/watch?v=rB83DpBJQsE

**Pseudodifferentialoperatoren**

* Die [Pseudodifferentialoperatoren](https://de.wikipedia.org/wiki/Pseudodifferentialoperator) sind eine Verallgemeinerung von Differentialoperatoren. Sie ermöglichen, aufbauend auf dem Grundbegriff der Distribution und auf der Fouriertransformation, ein ganz neues Verständnis für die Lösung partieller Differen- tialgleichungen.

**Integraloperator (mit Laplace- & Fourier-Transformation)**

* **Integral**: Der [**Volterraoperator**](https://de.wikipedia.org/wiki/Integraloperator#Volterraoperator) $\int_{0}^{t}$ zur Bildung des [bestimmten Integrals](https://de.wikipedia.org/wiki/Integralrechnung) (ist ein Beispiel fur einen [Integraloperator](https://de.wikipedia.org/wiki/Integraloperator). Operatoren wie diese, die einer Funktion eine Zahl zuordnen, nennt man [Funktional](https://de.wikipedia.org/wiki/Funktional).

  * Die Fourier-Transformation ${\mathcal {F}}$ ist z.B. ein **linearer Operator** (siehe unten). 
  
* Ein [linearer Integraloperator](https://de.wikipedia.org/wiki/Integraloperator) ist ein mathematisches Objekt aus der Funktionalanalysis. Dieses Objekt ist ein linearer Operator, der mit einer bestimmten Integralschreibweise mit einem Integralkern dargestellt werden kann.

* Einige Integraloperatoren nennt man traditionell eher [Integraltransformationen](https://de.wikipedia.org/wiki/Integraloperator#Integraltransformationen). Sie spielen zum Beispiel in der Signalverarbeitung eine wesentliche Rolle und dienen der besseren Handhabe und Analyse des Informationsgehaltes eines Signals x.

  * [Fourier-Transformation](https://de.m.wikipedia.org/wiki/Fourier-Transformation)

  * [Laplace-Transformation](https://de.m.wikipedia.org/wiki/Laplace-Transformation)

[**Fourier-Transformation**](https://de.m.wikipedia.org/wiki/Fourier-Transformation)
* is a coordinate transform to make something similar
* used for representing data and image compression
* approximate solutions for partial differential equations
* Fourier used it in heat equation
* Fourier transform is a coordinate transform to diagonalize the laplacian operator in heat equation. * this laplacian operator has eigenvalues and eigenfunctions like linear operators, and the eigenfunctions are sines & cosines with a particular frequency determined by the boundary conditions and the geometry of this object (i.e. a hot plate), and corresponding eigenvalues are the spatial frequencies 
* Make heat equation simpler by transforming it into an eigenvector eigenvalue coordinate system 
* SVD (singular value decomposition) is like a data driven extension of FFT
* approximate a function by a sum of sines and cosines of increasing high frequencies

* **Fast Fourier Transform** is a newer, but very important approach

* Fourier transform laid foundation for function spaces, like Hilbert space (pivotal for understanding quantum mechanics)
* Just like x and y axis form basis for vector spaces, **sine and cosine form an orthogonal basis for the function space** (of possible functions)

https://www.youtube.com/watch?v=spUNpyF58BY

https://www.youtube.com/watch?v=jNC0jxb0OxE&list=PLMrJAkhIeNNT_Xh3Oy0Y4LTj0Oxo8GqsC

#### **Types**

**Lineare & nicht-lineare Operatoren**

[Lineare Operatoren](https://de.wikipedia.org/wiki/Linearer_Operator): In Funktionalanalysis betrachtet man Eigenschaften von Abbildungen zwischen (unendlichdimensionalen) Banachräumen. **Lineare Abbildungen heißen [lineare Operatoren](https://de.wikipedia.org/wiki/Lineare_Abbildung)**, nichtlineare Abbildungen werden nichtlineare Operatoren genannt.

* Der Begriff linearer Operator wurde in der Funktionalanalysis (einem Teilgebiet der Mathematik) eingeführt und ist **synonym zum Begriff der linearen Abbildung**. Eine lineare Abbildung ist eine **strukturerhaltende Abbildung zwischen Vektorräumen über einem gemeinsamen Körper**. 

* Werden Vektorräume über dem Körper der reellen oder komplexen Zahlen betrachtet und sind diese mit einer Topologie versehen (lokalkonvexe Räume, normierte Räume, Banachräume), so spricht man vorzugsweise von linearen Operatoren.

* Die Bedeutung linearer Operatoren besteht darin, dass sie die lineare Struktur des unterliegenden Raumes respektieren, d. h., sie sind Homomorphismen zwischen Vektorräumen.

* **Anwendungen linearer Operatoren sind:**

  * Die Beschreibung von Koordinatentransformationen im dreidimensionalen Euklidischen Raum (**Spiegelung, Drehung, Streckung**) und der [Lorentztransformation](https://de.wikipedia.org/wiki/Lorentz-Transformation) in der vierdimensionalen Raumzeit durch Matrizen.

  * Die Darstellung von Observablen in der Quantenmechanik und die Beschreibung der Dynamik eines quantenmechanischen Systems durch seinen Hamilton-Operator H in der Schrödingergleichung.

  * Die Entwicklung von Lösungstheorien für Differential- und Integralgleichungen, siehe Sobolew-Raum und Distribution.

**Selbstadjungierte & symmetrische Operatoren**

* Der [selbstadjungierte Operator](https://de.wikipedia.org/wiki/Selbstadjungierter_Operator) ist eine Verallgemeinerung der selbstadjungierten Matrix.
* Siehe auch [Adjungierter_Operator](https://de.m.wikipedia.org/wiki/Adjungierter_Operator)

* ein [symmetrischer Operator](https://de.wikipedia.org/wiki/Symmetrischer_Operator) ist ein **linearer Operator** und wird in der Funktionalanalysis im Kontext **unbeschränkter Operatoren** betrachtet

* ein **beschränkter symmetrischer Operator ist ein selbstadjungierter Operator**.

* Sei $H$ ein Hilbertraum. Ein linearer Operator $T: D(T) \rightarrow H$ heißt symmetrisch, falls
$\langle T y, x\rangle=\langle y, T x\rangle$

**Beschränkte und unbeschränkte lineare Operatoren**

* Im Gegensatz zu endlichdimensionalen Räumen, wo lineare Operatoren stets beschränkt sind, tauchen bei unendlichdimensionalen Räumen auch unbeschränkte lineare Operatoren auf.

* unbeschränkte lineare Operatoren auf Hilberträumen:

  * lässt man oft auch Operatoren zu, deren Definitionsbereich (Domäne) lediglich ein Unterraum des betrachteten Raumes ist, so lässt man als Definitionsbereich auch einen Prähilbertraum als Teilraum eines Hilbertraums zu, präziser spricht man dann von dicht definierten unbeschränkten linearen Operatoren (s. u.). Der Operator wird als **partielle Abbildung** aufgefasst.

  * Differential- und Multiplikationsoperatoren sind i. A. unbeschränkt.

  * Die Darstellung von Observablen der Quantenmechanik erfordert unbeschränkte lineare Operatoren, da die den Observablen zugeordneten Operatoren i. A. unbeschränkt sind.

**Spezielle Klassen von Operatoren sind etwa [kompakte Operatoren](https://de.wikipedia.org/wiki/Kompakter_Operator) oder [Fredholm-Operatoren](https://de.wikipedia.org/wiki/Fredholm-Operator).**

**Operatortopologie**

* [Operatortopologien](https://de.m.wikipedia.org/wiki/Operatortopologie) sindverschiedene Topologien auf dem Raum der stetigen, linearen Operatoren auf einem Hilbertraum. 

#### **Operatoralgebren**

**Operatoralgebra (Banach-, C<sup>*</sup>- und W<sup>*</sup>-Algebren)**

* [Operatoralgebren](https://de.m.wikipedia.org/wiki/Operatoralgebra) sind  **Verallgemeinerungen der Matrizenalgebren** der linearen Algebra

* Sind $E, F, G$ normierte Räume und $A: E \rightarrow F$ und $B: F \rightarrow G$ stetige, lineare Operatoren, so ist auch deren Komposition ein stetiger, linearer Operator $B \circ A: E \rightarrow G,$ und für die Operatornormen gilt $\|B \circ A\| \leq\|B\| \cdot\|A\| .$ 

* Daher wird der Raum $L(E)$ der stetigen, linearen Operatoren von $E$ in sich mit der Komposition als Multiplikation zu einer normierten Algebra, die bei vollständiaem $E$ sogar eine Banachalgebra ist.

* Diese Algebren und ihre Unteralgebren nennt man Operatoralgebren, wobei der Fall, dass $E$ ein Hilbertraum ist, besonders intensiv untersucht wird. 

**Banachalgebren als Operatoralgebren**

* Jede normierte Algebra $\mathcal{A}$ kann als Operatoralgebra dargestellt werden. 

* Die sogenannte linksreguläre Darstellung von $\mathcal{A},$ die jedem Element $A$ den Operator $\ell_{A} \in L(\mathcal{A})$ zuordnet, wobei $\ell_{A}(B):=A B$, ist ein isometrischer Homomorphismus, falls $\mathcal{A}$ ein Einselement besitzt. Ist kein Einselement vorhanden, so adjungiere man eines.

* Welche Homomorphismen von einer Banachalgebra in eine Operatoralgebra existieren, wird in der Darstellungstheorie untersucht. Ein besonderes Interesse gilt dabei **Darstellungen auf Hilberträumen**, das heißt Homomorphismen in die Operatoralgebra über einem Hilbertraum, was zu den Begriffen **Von-Neumann-Algebra und C*-Algebra** führt.

* Operatoralgebren über Banachräumen, speziell über Hilberträumen, erlauben die Einführung zusätzlicher Topologien wie etwa die **starke oder schwache Operatortopologie**, wobei gerade letzterer wegen der Kompaktheit der Einheitskugel eine besondere Bedeutung zukommt.

**Banach-Algebren**

* [Banachalgebren](https://de.wikipedia.org/wiki/Banachalgebra), siehe auch [hier](https://mathepedia.de/Banachalgebra.html), sind mathematische **Objekte der Funktionalanalysis**, die einige wesentliche Eigenschaften mit bekannten Funktionenräumen (z. B. Mengen stetiger oder integrierbarer Funktionen) gemeinsam haben, und deren Verallgemeinerung sie daher sind.

* Eine Banachalgebra ist ein Vektorraum, in dem auch ein Produkt und eine Norm so definiert sind, dass gewisse Verträglichkeitsbedingungen erfüllt sind.

https://mathepedia.de/Banachalgebra.html

**C*-Algebren**

* [C*-Algebren](https://de.wikipedia.org/wiki/C*-Algebra) sind eine Abstraktion der beschränkten linearen Operatoren auf einem Hilbertraum, sie spielen daher in der mathematischen Beschreibung der Quantenmechanik eine Rolle.

* C<sup>*</sup>-Algebren **sind spezielle Banachalgebren**, bei denen ein enger Zusammenhang zwischen algebraischen und topologischen Eigenschaften besteht; die Kategorie der lokalkompakten Räume erweist sich als äquivalent zur Kategorie der kommutativen C<sup>*</sup>-Algebren, daher wird die **Theorie der C<sup>*</sup>-Algebren auch als nichtkommutative Topologie** angesehen. 

* Sofern eine solche nichtkommutative Topologie von einer Metrik induziert wird, wird diese durch das relativ neue Forschungsfeld der nichtkommutativen Geometrie erfasst,

**W*-Algebren (Von-Neumann-Algebra)**

* [Von-Neumann-Algebra](https://de.m.wikipedia.org/wiki/Von-Neumann-Algebra) ist eine 
mathematische Struktur in der Funktionalanalysis

* Jede Von-Neumann-Algebra ist eine C*-Algebra und somit auch eine Banachalgebra.

* Von-Neumann-Algebren enthalten sehr viele Orthogonalprojektionen; jeder Operator ist in der Normtopologie Limes von Linearkombinationen von Orthogonalprojektionen. Dies ist ein wesentlicher Unterschied zu den C*-Algebren

#### **Spektrum**

* Das [Spektrum](https://de.m.wikipedia.org/wiki/Spektrum_(Operatortheorie)) eines linearen Operators ist ein Begriff aus der Funktionalanalysis, einem Teilgebiet der Mathematik. 

* In der endlichdimensionalen linearen Algebra betrachtet man Endomorphismen, die durch Matrizen dargestellt werden, und ihre Eigenwerte. Die Verallgemeinerung ins Unendlichdimensionale wird in der Funktionalanalysis betrachtet. 

* **Das Spektrum eines Operators kann man sich als Menge verallgemeinerter Eigenwerte vorstellen. Diese werden Spektralwerte genannt.**

## **Funktionale & Distributionen**

#### **Lineare Funktionale**

* **Ein [Funktional](https://de.m.wikipedia.org/wiki/Funktional) bzw. [functional](https://en.m.wikipedia.org/wiki/Functional_(mathematics)) J[y] bildet eine <u>Funktion</u> y(x) auf eine <u>Zahl</u> ab.**

> Sei $V$ ein $\mathbb{K}$ -Vektorraum mit $\mathbb{K} \in\{\mathbb{R}, \mathbb{C}\} .$ Ein Funktional $T$ ist eine Abbildung $T: V \rightarrow \mathbb{K} .$

* eine Funktion, deren Definitionsmenge (Urbilder) als Teilmenge in einem Vektorraum V enthalten ist, während ihre Zielmenge in dem zugehörigen Skalarkörper liegt (zB von einer Fläche R<sup>2</sup> in eine Linie R)

* Exkurs Funktionalanalyse: Hier ist der untersuchte Vektorraum V zumeist ein Funktionenraum, also ein Vektorraum, dessen Elemente reell- oder komplexwertige Funktionen sind, wobei diesen durch Funktionale Skalare zugeordnet werden. 

* Als bedeutendes Beispiel eines solchen Funktionals kann etwa das [Lebesgue-Integral](https://de.m.wikipedia.org/wiki/Lebesgue-Integral) gelten.

**Unterschied zwischen Funktion und Funktional**

**Function:**
$\mathbb{R} \rightarrow f(x) \rightarrow \mathbb{R}$

**Functional:**
$f(x) \rightarrow J[f(x)] \rightarrow \mathbb{R}$

* Eine Funktion bildet Elemente eines Körpers auf andere ab, zum Beispiel. $f: \mathbb{R}^{3} \rightarrow \mathbb{R}^{2}$

* Ein Funktional hingegen ist ein Operator, der Elemente eines Vektorraumes $V$ auf den [Skalarkörper](https://de.wikipedia.org/wiki/Skalar_(Mathematik))
$K$ abbildet, über dem $V$ modelliert ist. Diese Elemente können zum Beispiel selber Funktionen sein
(wenn der Vektorraum ein Funktionenraum ist)

**Example of [Linear functional](https://de.m.wikipedia.org/wiki/Funktional#Lineare_Funktionale)**

* In den meisten Bereichen der Funktionalanalysis, etwa in der Theorie der topologischen Vektorräume, wird der Begriff Funktional (ohne weiteren Zusatz) als Synonym für lineare Funktionale benutzt. 

* Ein solches Funktional ist also definitionsgemäß eine Linearform, also **eine lineare Abbildung des Vektorraumes $V$ in seinen Skalarkörper $K$.**

* Die Menge all dieser Funktionale ist wiederum in natürlicher Form ein Vektorraum uber dem gleichen Körper $\mathbb{K}$, indem man für zwei Funktionale $f$ und $g$ über $V$ die Addition und Skalarmultiplikation punktweise definiert, $d .$ h.

> $
(f+g)(x):=f(x)+g(x) \quad(\lambda f)(x):=\lambda(f(x)), x \in V
$

* Der Vektorraum der linearen Funktionale auf dem Vektorraum $V$ wird der algebraische **Dualraum** genannt und oft mit $V^{*}$ bezeichnet.

**Linearform**

Stetige lineare Funktionale ([Linearform](https://de.m.wikipedia.org/wiki/Linearform)): 

* **In einem topologischen Vektorraum sind im Allgemeinen nicht alle linearen Funktionale steti**g. 

* Die stetigen linearen Funktionale innerhalb des algebraischen Dualraums, also die auf $V$ gegebenen stetigen [Linearformen](https://de.m.wikipedia.org/wiki/Linearform), bilden einen linearen Unterraum von $V^{*}$. **Dies ist der topologische Dualraum von $V$, der in der Funktionalanalysis einer der Hauptgegenstände ist.** 

* Er wird meist mit der Bezeichnung $V'$ gekennzeichnet, von einigen Autoren jedoch auch mit derselben Bezeichnung wie der algebraische Dualraum, also ebenfalls mit $V^{*}$.

* Im Kontext der Funktionalanalysis, das heißt im Falle eines topologischen 
$\mathbb {R}$ - oder $\mathbb {C}$-Vektorraums, sind die betrachteten Linearformen meistens [stetige lineare Funktionale](https://de.m.wikipedia.org/wiki/Funktional#Stetige_lineare_Funktionale).

* Es sei $K$ ein Körper und $V$ ein $K$ -Vektorraum. Eine Abbildung $f: V \rightarrow K$ heißt Linearform, wenn für alle Vektoren $x, y \in V$ und Skalare $\alpha \in K$ gilt:

1. $f(x+y)=f(x)+f(y)$ (**Additivität**);

2. $f(\alpha x)=\alpha f(x)$ (**Homogenität**).

* 'Linear form' is a more modern and abstract concept of 'functional': In modern linear algebra, functional refers to a linear mapping from a vector space $V$ into its field of scalars, i.e., it refers to an element of the dual space $V^{*}$. 

* Die Linearform ist ein Begriff aus dem mathematischen Teilgebiet der linearen Algebra. Man bezeichnet damit eine lineare Abbildung von einem Vektorraum in den zugrundeliegenden Körper.

* **Übergang zu Dualraum**: Die Menge aller Linearformen (= stetigen, linearen Abbildungen) über einem gegebenen Vektorraum $V$ bildet dessen Dualraum $V^{*}$ und damit selbst wieder in natürlicher Weise einen $K$ -Vektorraum.

**[One-form](https://en.m.wikipedia.org/wiki/One-form) (Pfaffsche Form)**

* In linear algebra, a one-form on a vector space is the same as a linear functional (linear form) on the space. 

* The usage of one-form in this context usually distinguishes the one-forms from higher-degree multilinear functionals on the space. For details, see [linear functional / linear form](https://en.m.wikipedia.org/wiki/Linear_form).

#### **Dualraum**

Beim Studium normierter Räume ist die Untersuchung des Dualraumes wichtig. Der Dualraum besteht aus allen **stetigen linearen** Funktionen vom **normierten** Raum in seinen Skalarkörper, also in die reellen oder komplexen Zahlen. Der Bidual, also der Dualraum des Dualraums, muss nicht isomorph zum ursprünglichen Raum sein, aber es gibt stets einen natürlichen Monomorphismus von einem Raum in seinen Bidual. Ist dieser spezielle Monomorphismus auch surjektiv, dann spricht man von einem reflexiven Banachraum.



Dualraum: https://youtu.be/2vvjrBbcTZU

* In den meisten Bereichen der Funktionalanalysis, etwa in der Theorie der [topologischen
Vektorräume](https://de.m.wikipedia.org/wiki/Topologischer_Vektorraum), wird der Begriff Funktional (ohne weiteren Zusatz) als Synonym für lineare Funktionale
benutzt. 

* Ein solches Funktional ist also definitionsgemäß eine [Linearform](https://de.m.wikipedia.org/wiki/Linearform), also eine lineare
Abbildung des Vektorraumes $V$ in seinen Skalarkörper $\mathbb{K}$. 

* Die Menge all dieser Funktionale ist
wiederum in natürlicher Form ein Vektorraum über dem gleichen Körper $\mathbb{K}$, indem man für zwei
Funktionale $f$ und $g$ über $V$ die Addition und Skalarmultiplikation punktweise definiert, d. h.


> $
(f+g)(x):=f(x)+g(x) \quad(\lambda f)(x):=\lambda(f(x)), x \in V
$

Der Vektorraum der linearen Funktionale auf dem Vektorraum $V$ wird der algebraische Dualraum genannt und oft mit $V^{*}$ bezeichnet.

**Beispiele von Dualräumen**

Für den Vektorraum $V=\mathbb{R}^{n}$ **ist der Dualraum kanonisch isomorph zum Vektorraum selbst**, $\mathrm{d} .$ h. $V \cong V$ Der kanonische Isomorphismus $I: \mathbb{R}^{n} \rightarrow\left(\mathbb{R}^{n}\right)^{*}$ wird dabei über das [Standardskalarprodukt](https://de.m.wikipedia.org/wiki/Standardskalarprodukt) vermittelt:

$
(I(x))(y):=\langle x, y\rangle=\sum_{i=1}^{n} x_{i} \cdot y_{i}
$

Für den Vektorraum $V=\mathbb{C}^{n}$ gilt ähnliches wie im ersten Fall, allerdings ist die kanonische Abbildung $I: \mathbb{C}^{n} \rightarrow\left(\mathbb{C}^{n}\right)^{*}$ in diesem Fall semilinear:

$
(I(x))(y):=\langle x, y\rangle=\sum_{i=1}^{n} \overline{x_{i}} \cdot y_{i}
$

Der Dualraum ist in diesem Fall also gleich groB, hat aber bezüglich der kanonischen Abbildung eine
andere Skalarmultiplikation. Im Sinne der linearen Algebra sagt man auch: Der Dualraum ist kanonisch
isomorph zum komplex konjugierten Vektorraum.

*Beispiele topologischer Dualräume*

* Für endlichdimensionale Vektorräume gibt es eine natürliche Topologie ([Normtopologie](https://de.m.wikipedia.org/wiki/Normtopologie)), die aus der euklidischen Norm hervorgeht (genauer gesagt: aus einer beliebigen euklidischen Norm, wenn man eine Basis wählt). Dies ist gerade die Topologie, die der normalen Standard-Analysis zugrunde liegt, und in dieser ist jedes lineare Funktional stetig. Das heißt, der algebraische Dualraum ist gleich dem topologischen Dualraum.

* Im unendlichdimensionalen Fall ist der topologische Dualraum (fast) immer ein echter Teilraum des algebraischen Dualraumes.

* In **normierten Vektorräumen ist ein Funktional $f$ genau dann stetig, wenn es beschränkt ist**, das heißt $\sup _{\|x\| \leq 1}|f(x)|<\infty$. Der topologische Dualraum ist dann automatisch ein Banachraum mit der oben angegebenen Supremumsnorm.

* In Hilberträumen ist der topologische Dualraum kanonisch mit dem Ursprungsraum identifizierbar (Darstellungssatz von Fréchet-Riesz). Die Identifikation erfolgt wie im endlichdimensionalen Fall über das Skalarprodukt: $(I(y)):=\langle\cdot, y\rangle$

* Der topologische Dualraum des Raumes der unendlich oft stetig differenzierbaren Funktionen mit kompaktem Träger auf der reellen Achse (die so genannten Testfunktionen) mit einer bestimmten (hier nicht näher erklärten) Topologie wird als Raum der [Distributionen](https://de.m.wikipedia.org/wiki/Distribution_(Mathematik)) bezeichnet. In diesem Raum liegt auch das weiter oben genannte Beispiel des Dirac-Delta-Funktionals.

#### **Non-linear Functional**

**Example of [Non-linear functional](https://de.m.wikipedia.org/wiki/Funktional#Nichtlineare_Funktionale)**

* Nichtlineare Funktionale traten historisch erstmals in der **Variationsrechnung** auf. 

* Ihr Studium unterscheidet sich grundlegend von dem der oben beschriebenen linearen Funktionale. In der Variationsrechnung setzt man es sich beispielsweise zum Ziel, die Extremalpunkte solcher Funktionalpunkte zu bestimmen. 

* Zu diesem Zweck benötigt man eine Verallgemeinerung des Ableitungsbegriffs der mehrdimensionalen Analysis, d. h. eine Definition des Differentials des Funktionals. In der Variationsrechnung und in den Anwendungen ist dieses Differential unter dem Namen Variationsableitung bekannt, mathematisch präzisiert wird der Begriff z. B. durch die Fréchet-Ableitung und die Gateaux-Ableitung.

[**Functional derivative**](https://en.m.wikipedia.org/wiki/Functional_derivative)

* Functional derivatives are used in Lagrangian mechanics. They are derivatives of functionals: i.e. they carry information on how a functional changes when the input function changes by a small amount.

[**Functional integration**](https://en.m.wikipedia.org/wiki/Functional_integration)

* Richard Feynman used functional integrals as the central idea in his sum over the histories formulation of quantum mechanics. This usage implies an integral taken over some function space.

#### **Distribution & Test Function**

https://mathepedia.de/Distributionen.html

* Eine Distribution bezeichnet eine **besondere Art eines Funktionals** (=eine Funktion, die anderen Funktionen eine Zahl zuordnet) im Bereich der partiellen Differentialgleichungen

* Es gibt partielle Differentialgleichungen, die hinreichend oft differenzierbare oder gar keine klassischen Lösungen haben, aber Lösungen im distributionellen Sinn. 

* Daher können Distributionen als eine **Verallgemeinerung von Funktionen** angesehen werden. (z.B. kann man jede Distribution differenzieren.)

* Problem: Losungen mit 'Knicken' in Differentialgleichungen. Begriff einer Ableitung ist zu stark, man muss also die Losungen verallgemeinern.

* zB Ableitung der [Heaviside Funktion](https://de.wikipedia.org/wiki/Heaviside-Funktion) - 

  * Nutzung der Delta Funktion. Delta Funktion ist Null ausserhalb der kritischen Stelle Null.
  * Aber: delta ist keine Funktion im ublichen Sinn. Delta ist Null fast uberall (bez. des Lebesgue Masses)
  * Das heisst jegliches Integral ist Null.

* Losung: Delta wird als 'Distribution' definiert (oder auch: 'verallgemeinerte Funktion'). Funktionen fasst man jetzt als eine Dichte auf (zB Massedichte auf einem eindimensionalen Stab). Delta-Funktion ist dann eine singulare Dichte (Punktmasse).

https://www.youtube.com/watch?v=c5WYrQK_7ls

* Testfunktionen: Miss die Dichte aus mit einem 'virtuellen Messgerat' unterhalb einer Funktion. Sollten stetig und lokalisiert sein. 



Sei $\Omega \subset \mathbb{R}^{n}$ eine offene, nichtleere Menge. Eine Distribution $T$ ist ein stetiges und lineares Funktional auf dem Raum der [Testfunktionen](https://de.m.wikipedia.org/wiki/Testfunktion) $\mathcal{D}(\Omega)$.

In anderen Worten eine Abbildung $T: \mathcal{D}(\Omega) \rightarrow \mathbb{C}$ bzw. $\mathbb{R},$ so dass für alle $\phi_{1}, \phi_{2} \in \mathcal{D}(\Omega)$ und $\lambda \in \mathbb{C}$ gilt $T\left(\phi_{1}+\lambda \phi_{2}\right)=T\left(\phi_{1}\right)+\lambda T\left(\phi_{2}\right)$ und $T\left(\phi_{n}\right) \rightarrow T(\phi)$ wann immer $\phi_{n} \rightarrow \phi$ in $\mathcal{D}(\Omega)$

**Temperierte Distribution**

* Eine [temperierte Distribution](https://de.m.wikipedia.org/wiki/Temperierte_Distribution) ist ein Spezialfall einer Distribution

* Laurent Schwartz führte 1947 den Raum der temperierten Distributionen ein, um die Fourier-Transformation in seine Distributionentheorie integrieren zu können.

**Delta-Distribution** ([Dirac-Funktion](https://de.m.wikipedia.org/wiki/Delta-Distribution))
* ist eine spezielle irreguläre Distribution mit kompaktem Träger. Sie hat in der Mathematik und Physik grundlegende Bedeutung. Ihr übliches Formelsymbol ist δ (kleines Delta).

https://de.m.wikipedia.org/wiki/Distribution_(Mathematik)

**Test Function**

* Als Testfunktionen bezeichnet man gewisse Typen von Funktionen, die in der Distributionentheorie eine wesentliche Rolle spielen. Üblicherweise **fasst man Testfunktionen eines bestimmten Typs zu einem Vektorraum** zusammen. 

* **Die zugehörigen Distributionen sind dann lineare Funktionale auf diesen Vektorräumen**. Ihr Name rührt daher, dass man die Distributionen (im Sinne linearer Abbildungen) auf die Testfunktionen anwendet und dadurch testet

* Es gibt verschiedene Arten von Testfunktionen. In der mathematischen Literatur werden häufig der Raum der glatten Funktionen mit kompaktem Träger oder der Schwartz-Raum als Testfunktionenraum bezeichnet.

formulier problem in variationell (sobolove räume), und dann Eigenschaften von Testfunktionen ausnutzen

https://www.youtube.com/watch?v=beu28z_q3po

https://en.wikipedia.org/wiki/Bump_function

https://de.m.wikipedia.org/wiki/Testfunktion

https://en.m.wikipedia.org/wiki/Test_functions_for_optimization

**Schwarz-Raum**

https://de.m.wikipedia.org/wiki/Schwartz-Raum

## **Variationsrechnung**

https://de.wikipedia.org/wiki/Fluiddynamik

formulier problem in variationelle formulierung (dann bist du in sobolove räume), und dann Eigenschaften von Testfunktionen ausnutzen

#### **Gegenstand**

* Die [Variationsrechnung](https://de.wikipedia.org/wiki/Variationsrechnung) ist eine **Erweiterung der Funktionalanalysis und beschaeftigt sich mit <u>nichtlinearen Funktionalen</u>** (in der Funktionalanalysis sind es linear Funktionale)

* The [calculus of variations](https://en.m.wikipedia.org/wiki/Calculus_of_variations) is a field that **uses variations, which are small changes in functions and functionals, to find maxima and minima of functionals**: mappings from a set of functions to the real numbers. Functionals are often expressed as definite integrals involving functions and their derivatives. Functions that maximize or minimize functionals may be found using the Euler–Lagrange equation of the calculus of variations.

* In calculus of variations we are **NOT concerned with finding fix points of functions (like local maxima in a function), but rather fix points of functionals.**


* dann führt eine Variation der Wirkung: https://de.m.wikipedia.org/wiki/Feldtheorie_(Physik)#Formalismus

**Variational Principle**

* a [variational principle](https://en.m.wikipedia.org/wiki/Variational_principle) is one that enables a problem to be solved using calculus of variations, which concerns finding such functions which optimize the values of quantities that depend upon those functions. 

* For example, the problem of determining the shape of a hanging chain suspended at both ends—a catenary—can be solved using variational calculus, and in this case, the variational principle is the following: The solution is a function that minimizes the gravitational potential energy of the chain.

* Any physical law which can be expressed as a variational principle describes a self-adjoint operator. These expressions are also called Hermitian. Such an expression describes an invariant under a Hermitian transformation.

**Variation der Elemente**

* die [Variation der Elemente](https://de.wikipedia.org/wiki/Variation_der_Elemente) ist eine im 19. Jahrhundert entwickelte Methode zur genauen Bahnbestimmung von Himmelskörpern. Sie dient bis heute zur Modellierung von [Bahnstörungen](https://de.wikipedia.org/wiki/Bahnstörung).

**Brachistochrone & Tautochronie**

Johann I Bernoulli hat sich mit dem **Problem des schnellsten Falles** beschäftigt. Im Jahre 1696 fand er schließlich die Lösung in der **Brachistochrone**. Heute sieht man dies oft als die **Geburtsstunde der Variationsrechnung**.

[**Brachistochrone Curve**](https://en.wikipedia.org/wiki/Brachistochrone_curve) (in rot): Der Körper gleitet auf einer solchen Bahn schneller zum Ziel als auf jeder anderen Bahn, beispielsweise auf einer geradlinigen, obwohl diese kürzer ist.


![vv](https://upload.wikimedia.org/wikipedia/commons/6/63/Brachistochrone.gif)

**Tautochronie** der Brachistochrone – von jedem Startpunkt auf der Kurve erreichen die Kugeln das „Ziel“ gleichzeitig.

![ff](https://upload.wikimedia.org/wikipedia/commons/b/bd/Tautochrone_curve.gif)

**Gâteaux-Differential**

* [Gâteaux-Differential](https://de.wikipedia.org/wiki/Gâteaux-Differential) ist eine **Verallgemeinerung des gewöhnlichen Differentiationsbegriffes** dar, indem es die Richtungsableitung auch in unendlichdimensionalen Räumen definiert.

#### **Euler–Lagrange Equation**

**Hamilton'sches Prinzip (Wirkungsfunktional)**

* Das [Hamilton'sche Prinzip](https://de.wikipedia.org/wiki/Hamiltonsches_Prinzip) zeichnet tatsachlich durchlaufene Bahnen dadurch aus, dass bei ihnen die Wirkung S[q] (verglichen mit anderen Bahnen) ein Minimum annimmt.

* Das Hamiltonsche Prinzip der Theoretischen Mechanik ist ein **Extremalprinzip**. Physikalische Felder und Teilchen nehmen danach für eine bestimmte Größe einen extremalen (d. h. größten oder kleinsten) Wert an. Diese Bewertung nennt man Wirkung, mathematisch ist die Wirkung ein Funktional, daher auch die Bezeichnung **Wirkungsfunktional**.

* Die Wirkung erweist sich in vielen Fällen nicht als minimal, sondern nur als **„stationär“** (d. h. extremal). Deshalb wird das Prinzip von manchen Lehrbuchautoren auch das Prinzip der **stationären Wirkung** genannt. Manche Autoren nennen das Hamiltonsche Prinzip auch **'Prinzip der kleinsten Wirkung'**, was jedoch – wie oben ausgeführt – nicht präzise ist.

* Hamilton's principle states that the true evolution of a physical system is a solution of the functional equation:

> $\frac{\delta \mathcal{S}}{\delta \mathbf{q}(t)}=0$

* That is, the system takes a path in configuration space for which the action is stationary, with fixed boundary conditions at the beginning and the end of the path.

![ff](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/Least_action_principle.svg/500px-Least_action_principle.svg.png)

*As the system evolves, q traces a path through configuration space (only some are shown). The path taken by the system (red) has a stationary action (δS = 0) under small changes in the configuration of the system (δq).*

Richard Feynman zeigte in den 1940ern, dass sich das Hamiltonsche Prinzip in der Quantenfeldtheorie gerade dadurch ergibt, dass alle möglichen Pfade (auch die nicht zielgerichteten) zulässig sind und aufintegriert werden. Dabei überlagern sich Pfade mit extremaler Wirkung konstruktiv und davon abweichende destruktiv, so dass die Natur schließlich zielgerichtet erscheint.

**Langrange Formalismus**

Eingeführte Formulierung der klassischen Mechanik, in der die Dynamik eines Systems durch **eine einzige skalare Funktion, die Lagrange-Funktion**, beschrieben wird. Der Formalismus ist (im Gegensatz zu der newtonschen Mechanik, die a priori nur in Inertialsystemen gilt) auch in beschleunigten Bezugssystemen gültig. Der Lagrange-Formalismus ist invariant gegen Koordinatentransformationen.

Der [Langrange Formalismus](https://de.wikipedia.org/wiki/Lagrange-Formalismus) ist eine mögliche (von vielen!) Formulierung der klassischen Mechanik. Hier wird die Dynamik eines Systems durch die Langrange-Funktion L(${\boldsymbol{q}}$, $\dot{\boldsymbol{q}}$, $t$) beschrieben:

* ${\boldsymbol{q}}$ = (q1, q2, ..., qw) - allgemeine Koordinaten im Raum
* $\dot{\boldsymbol{q}}$ = (d / dt) q ... ($\dot{\boldsymbol{q}}$1, $\dot{\boldsymbol{q}}$2.. $\dot{\boldsymbol{q}}$n) - Vektor der Wirkung (allgemeine Geschwindigkeiten)
* $t$ - Zeit (Die explizite Zeitabhängige berücksichtigt externe, zeitabhängige Faktoren (z.B. Magnet-Felder etc). Wird auf Null gesetzt, denn bei einem Wechsel in die mikroskopische Theorie verschwindet die Zeit)

> $L_{x}(q, \dot{q}, t)$ wird zu: $L_{x}(q, \dot{q})$ = $T$<sub>kin</sub> - $V$<sub>pot</sub>

https://www.youtube.com/watch?v=V8CR4GVg5Io&t=301s

https://www.youtube.com/watch?v=EceVJJGAFFI

The Euler–Lagrange equation is an equation satisfied by a function q of a real argument t, which is a stationary point of the functional:

> $S(\boldsymbol{q})=\int_{a}^{b} L(t, \boldsymbol{q}(t), \dot{\boldsymbol{q}}(t)) \mathrm{d} t$

* ${\boldsymbol{q}}$ - Koordinaten im Raum
* $\dot{\boldsymbol{q}}$ - Vektor der Wirkung
* t - Zeit (wird auf Null gesetzt, den bei einem Wechsel in die mikroskopische Theorie verschwindet die Zeit)

The Euler–Lagrange equation, then, is given by

> $L_{x}(t, q(t), \dot{q}(t))-\frac{\mathrm{d}}{\mathrm{d} t} L_{v}(t, q(t), \dot{q}(t))=0$

* partial derivative of one dimension, then second dimension and then time

**Euler–Lagrange Equation**

https://www.youtube.com/watch?v=V0wx0JBEgZc

* the [Euler equation](https://en.m.wikipedia.org/wiki/Euler–Lagrange_equation) is a **second-order partial differential** equation whose **solutions are the functions for which a given functional is stationary**.

* Because **a differentiable functional is stationary at its local extrema**, the Euler–Lagrange equation is useful for solving optimization problems in which, given some functional, one seeks the function minimizing or maximizing it. 

* This is analogous to [Fermat's theorem](https://en.m.wikipedia.org/wiki/Fermat%27s_theorem_(stationary_points)) in calculus, stating that at any point where a differentiable function attains a local extremum its derivative is zero.

* In Lagrangian mechanics, according to [Hamilton's principle](https://en.m.wikipedia.org/wiki/Hamilton%27s_principle) of stationary action, the evolution of a physical system is described by the solutions to the Euler equation for the action of the system. In this context Euler equations are usually called Lagrange equations. In classical mechanics, it is equivalent to Newton's laws of motion, but it has the advantage that it takes the same form in any system of generalized coordinates, and it is better suited to generalizations.

  * Hamilton's principle is William Rowan Hamilton's formulation of the [principle of stationary action](https://en.m.wikipedia.org/wiki/Principle_of_least_action) (also called: 'Principle of least action'). It states that the **dynamics of a physical system are determined by a variational problem for a functional based on a single function**, the Lagrangian, which may contain all physical information concerning the system and the forces acting on it. The variational problem is equivalent to and allows for the derivation of the differential equations of motion of the physical system


https://de.wikipedia.org/wiki/Variationsrechnung#Euler-Lagrange-Gleichung;_Variationsableitung;_weitere_notwendige_bzw._hinreichende_Bedingungen

#### **Fundamentallemma & -satz**

**Fundamentallemma der Variationsrechnung**

https://de.wikipedia.org/wiki/Fundamentallemma_der_Variationsrechnung

**Fundamentalsatz der Variationsrechnung**

* Fundamental Theorem of the Calculus of Variations - [Fundamentalsatz der Variationsrechnung](https://de.wikipedia.org/wiki/Fundamentalsatz_der_Variationsrechnung)

* eng verwandt mit dem [weierstraßschen Satz vom Minimum](https://de.wikipedia.org/wiki/Satz_vom_Minimum_und_Maximum)

* Er behandelt die in der Variationsrechnung zentrale Frage, unter welchen Bedingungen reellwertige Funktionale ein Minimum annehmen.

## **Vektoranalysis**

* Die [Vektoranalysis](https://de.m.wikipedia.org/wiki/Vektoranalysis) ist ein **Teilgebiet der Tensoranalysis**, beschäftigt sich hauptsächlich mit **Vektorfeldern in zwei oder mehr Dimensionen** 

* Die Vektoranalysis **verallgemeinert die Differential- und der Integralrechnung**.

* Betrachtet werden Vektorfelder, die jedem Punkt des Raumes einen Vektor zuordnen, und [Skalarfelder](https://de.m.wikipedia.org/wiki/Skalarfeld), die jedem Punkt des Raumes einen Skalar zuordnen
  * Skalarfeld $\varphi$: eine Funktion, die jedem Punkt eines Raumes eine reelle Zahl (Skalar) zuordnet, z. B. eine Temperatur, Luftdruck.
  * Wichtige Operationen im Zusammenhang mit Skalarfeldern sind: [Gradient](https://de.m.wikipedia.org/wiki/Gradient_(Mathematik)) eines Skalarfeldes, der ein Vektorfeld ist.
[Richtungsableitung](https://de.m.wikipedia.org/wiki/Richtungsableitung) eines Skalarfeldes. (Eine Verallgemeinerung der Richtungsableitung auf unendlichdimensionale Räume ist das [Gâteaux-Differential](https://de.m.wikipedia.org/wiki/G%C3%A2teaux-Differential).)
  * Ein Skalarfeld ist das einfachste [Tensorfeld](https://de.m.wikipedia.org/wiki/Tensorfeld).

*  Die Temperatur eines Swimmingpools ist ein Skalarfeld: Jedem Punkt wird der Skalarwert seiner Temperatur zugeordnet. Die Wasserbewegung entspricht dagegen einem Vektorfeld, da jedem Punkt ein Geschwindigkeitsvektor zugeordnet wird, der Betrag und Richtung hat.

#### **Integralsätze**

[Integralsatz von Gauß](https://de.m.wikipedia.org/wiki/Gaußscher_Integralsatz)

* Im Folgenden sei das „Integrationsvolumen“ V n-dimensional. Das [Volumenintegral](https://de.m.wikipedia.org/wiki/Volumenintegral) über den Gradienten einer skalaren Größe $\phi$, kann dann in ein [Oberflächenintegral](https://de.m.wikipedia.org/wiki/Oberflächenintegral) (bzw. Hyperflächenintegral) über den Rand dieses Volumens umgewandelt werden:

[Satz von Stokes](https://de.m.wikipedia.org/wiki/Satz_von_Stokes)

* sehr grundlegenden Satz über die Integration von Differentialformen, der den Hauptsatz der Differential- und Integralrechnung erweitert

*  eine Verbindungslinie von der Differentialgeometrie zur Algebraischen Topologie eröffnet. 

* Dieser Zusammenhang wird durch den Satz von de Rham beschrieben, für den der Satz von Stokes grundlegend ist.

#### **Fundamentalzerlegung**

* der [Helmholtzscher Zerlegungssatz](https://de.m.wikipedia.org/wiki/Helmholtz-Theorem) ist der Fundamentalsatz der Vektoranalysis. Beschreibt den allgemeinen Fall. Er sagt aus, dass sich jedes Vektorfeld $\vec{F}$ als eine Überlagerung eines Quellenanteils $\vec{F}_{Q}$ und eines Wirbelanteils $\vec{F}_{W}$ beschreiben lässt.

**[Navier-Cauchy-Gleichungen](https://de.m.wikipedia.org/wiki/Navier-Cauchy-Gleichungen)**

* mathematisches Modell der Bewegung elastischer Festkörper

## **Tensoranalysis**

* Die [Tensoranalysis](https://de.m.wikipedia.org/wiki/Tensoranalysis) ist ein Teilgebiet der Differentialgeometrie beziehungsweise der Differentialtopologie.

* In der Tensoranalysis wird **das Verhalten von geometrischen Differentialoperatoren auf Tensorfeldern untersucht**.

* Sie ist eine **Verallgemeinerung der Vektoranalysis**

* Zum Beispiel kann der Differentialoperator Rotation in diesem Kontext auf n Dimensionen verallgemeinert werden. 

* Zentrale Objekte der Tensoranalysis sind Tensorfelder. Es wird untersucht, wie Differentialoperatoren auf diesen Feldern wirken.

#### **Tensorfeld**

* Tensorfelder sind Funktionen, **die jedem Punkt einen Tensor zuordnen** (Tensor meint in diesem Fall ein rein algebraisches Objekt)

* Tensorfelder werden auf ihre analytischen Eigenschaften untersucht (zB differenziert). Man erhält durch Differenzieren eines Tensorfeldes wieder ein Tensorfeld. Tensorfelder sind besondere glatte Abbildungen, die in Tensorbündel hinein abbilden (siehe unten).

* Sei $M$ eine differenzierbare Mannigfaltigkeit. Ein [Tensorfeld](https://de.m.wikipedia.org/wiki/Tensorfeld) vom Typ (r,s) ist ein glatter [Schnitt](https://de.m.wikipedia.org/wiki/Schnitt_(Faserbündel)) im Tensorbündel $T_{s}^{r}(M)$. 

  * Ein Tensorfeld ist also ein glattes Feld $M \rightarrow T_{s}^{r}(M),$ welches jedem Punkt der Mannigfaltigkeit einen (r,s)-Tensor zuordnet. 

  * Die Menge der Tensorfelder wird oft mit $\Gamma^{\infty}\left(T_{s}^{r}(M)\right)$ bezeichnet.

#### **Tensordichte**

* [Tensordichte](https://de.m.wikipedia.org/wiki/Tensordichte) ist die Quantitätsgröße eines Tensorfeldes

* die Tensordichte ist eine **Verallgemeinerung der Tensorfelder** in der Tensoranalysis 

* wurde eingeführt, um den „Unterschied zwischen Quantität und Intensität, soweit er physikalische Bedeutung hat“, zu erfassen: „die Tensoren sind die Intensitäts-, die Tensordichten die Quantitätsgrößen“. 

* eine **Tensordichte** ordnet einem Koordinatensystem ein Tensorfeld derart zu, dass es bei einem Koordinatenwechsel mit dem Absolutbetrag der Funktionaldeterminante multipliziert wird. Eine Tensordichte der Stufe null ist demnach eine skalare Dichte, deren Integral gemäß dem Transformationssatz eine Invariante liefert.

#### **Vektorbündel & Tensorbündel**

* [Vektorbündel](https://de.m.wikipedia.org/wiki/Vektorbündel) oder manchmal auch Vektorraumbündel sind Familien von Vektorräumen, die **durch die Punkte eines topologischen Raumes parametrisiert sind**.

* Vektorbündel gehören damit auch zu den [Faserbündeln](https://de.m.wikipedia.org/wiki/Faserbündel). Remind: Faser ist ein Urbild von einem Element ("Faser der Abbildung über einem Element") - surjektiv! kann also mehrere Elemente im Urbild haben. Daher Faser $\mathbb{R}$<sup>2</sup> zu Punkt auf $\mathbb{R}$ (siehe [hier](https://de.m.wikipedia.org/wiki/Vektorbündel) die Illustration:


![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Vectorbundle.svg/320px-Vectorbundle.svg.png)

*Illustration des Vektorbündels $(E, B, \pi)$. Hier ist der **Totalraum** $E=\mathbb{R}^{2}$ und der **Basisraum** $B=\mathbb{R} .$ Die Abbildung $\pi: E \rightarrow B$ projiziert jede Gerade $E_{x}$ auf den Punkt $x$. Der Raum $E_{x}=\{p \in E \mid \pi(p)=x\}$ wird **Faser über $x$** genannt. Außerdem ist der Totalraum $E$ die Vereinigung aller Fasern.* (Comment: also Totalraum E ist Urbild mit Faser und Basisraum B ist Zielbild mit Element das von der Faser stammt)

* [Tensorbündel](https://de.m.wikipedia.org/wiki/Tensoranalysis#Tensorbündel) ist ein bestimmtes Vektorbündel. Tensorfelder sind dann besondere glatte Abbildungen, die in dieses Vektorbündel hinein abbilden.

#### **Schnitt (Faserbündel)**

* [Schnitte](https://de.m.wikipedia.org/wiki/Schnitt_(Faserbündel)) sind Abbildungen, welche in der algebraischen Topologie, insbesondere in der Homotopietheorie, untersucht werden. Insbesondere interessiert man sich dafür, unter welchen Bedingungen solche Abbildungen existieren. 

* Das bekannteste Beispiel von Schnitten sind die [**Differentialformen**](https://de.m.wikipedia.org/wiki/Differentialform).

* Ein Schnitt kann als **Verallgemeinerung des Graphen einer Funktion** aufgefasst werden. 



![cc](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Bundle_section.svg/187px-Bundle_section.svg.png)

*Die Abbildung s ist ein Schnitt in einem Faserbündel $p: E \rightarrow B$. Dieser Schnitt s erlaubt es, den Basisraum $B$ mit dem Teilraum $s(B)$ von $E$ zu identifizieren.*

## **Differentialgeometrie**

* Die [Differentialgeometrie](https://de.wikipedia.org/wiki/Differentialgeometrie) stellt die Synthese von Analysis und Geometrie dar.

https://en.wikipedia.org/wiki/Cycloid

https://de.wikipedia.org/wiki/Zentralprojektion

https://de.wikipedia.org/wiki/Projektive_Geometrie

https://de.wikipedia.org/wiki/Synthetische_Geometrie

https://de.wikipedia.org/wiki/Affiner_Raum

https://de.wikipedia.org/wiki/Euklidischer_Raum

https://de.wikipedia.org/wiki/Lorentz-Transformation

https://de.wikipedia.org/wiki/Minkowski-Raum

#### **Vektorfeld**

* In der mehrdimensionalen Analysis und der Differentialgeometrie ist ein [Vektorfeld](https://de.m.wikipedia.org/wiki/Vektorfeld) eine Funktion, die jedem Punkt eines Raumes einen Vektor zuordnet. 

* Das duale Konzept zu einem Vektorfeld ist eine Funktion, die jedem Punkt eine [Linearform](https://de.m.wikipedia.org/wiki/Linearform), [zB [stetige lineare Funktionale](https://de.m.wikipedia.org/wiki/Funktional#Stetige_lineare_Funktionale)] zuordnet, eine solche Abbildung wird [pfaffsche Form](https://de.m.wikipedia.org/wiki/Pfaffsche_Form) (One-Form) genannt.

#### **Differentialformen**

* [Differentialform](https://de.m.wikipedia.org/wiki/Differentialform) erlauben eine koordinatenunabhängige Integration auf allgemeinen orientierten differenzierbaren Mannigfaltigkeiten.

* Beispiele:

  * Glatte Funktionen sind 0-Formen.
  * Pfaffsche Formen sind 1-Formen.

#### **Differenzierbare Mannigfaltigkeit**

* [Differenzierbare Mannigfaltigkeiten](https://de.m.wikipedia.org/wiki/Differenzierbare_Mannigfaltigkeit) sind Objekte in der Differenzialtopologie, und ein Oberbegriff für Kurven, Flächen und andere geometrische Objekte, die – aus der Sicht der Analysis – lokal aussehen wie ein [euklidischer Raum](https://de.m.wikipedia.org/wiki/Euklidischer_Raum).

## **Funktionentheorie**

#### **Holomorphe Funktion**

* [Holomorphe Funktionen](https://de.m.wikipedia.org/wiki/Holomorphe_Funktion) sind an jedem Punkt komplex differenzierbar